In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Cargar los datos
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# Procesar la columna 'Weight' y convertirla a numérico
train_df['Weight'] = train_df['Weight'].str.replace('kg', '').astype(float)
test_df['Weight'] = test_df['Weight'].str.replace('kg', '').astype(float)

# Procesar la columna 'Ram' y convertirla a numérico
train_df['Ram'] = train_df['Ram'].str.replace('GB', '').astype(int)
test_df['Ram'] = test_df['Ram'].str.replace('GB', '').astype(int)

# Extraer la resolución de pantalla
train_df[['res_width', 'res_height']] = train_df['ScreenResolution'].str.extract(r'(\d{3,4})x(\d{3,4})').astype(float)
test_df[['res_width', 'res_height']] = test_df['ScreenResolution'].str.extract(r'(\d{3,4})x(\d{3,4})').astype(float)

# Procesar la columna 'Memory' y convertirla a numérico
def parse_memory(memory_str):
    if isinstance(memory_str, str):
        memory_str = memory_str.strip().lower()
        if 'gb' in memory_str:
            return int(float(memory_str.split('gb')[0].strip()))
        elif 'tb' in memory_str:
            return int(float(memory_str.split('tb')[0].strip()) * 1024)
    return 0

train_df['Memory'] = train_df['Memory'].apply(parse_memory)
test_df['Memory'] = test_df['Memory'].apply(parse_memory)

# Seleccionar las características que usaremos para el modelo
features_to_use_improved = ['Inches', 'Ram', 'Weight', 'Memory', 'res_width', 'res_height']

# Crear los conjuntos de entrenamiento y prueba
X_train_improved = train_df[features_to_use_improved]
X_test_improved = test_df[features_to_use_improved]

# Separar la variable objetivo (target)
y_train = train_df['Price_euros']

# Dividir los datos en conjunto de entrenamiento y validación
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_improved, y_train, test_size=0.2, random_state=42)

# Aplicar la transformación logarítmica para hacer la distribución más normal
y_train_log = np.log1p(y_train_split)
y_val_log = np.log1p(y_val_split)


In [7]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# Crear el modelo XGBoost
xgb_model_simple = XGBRegressor(
    n_estimators=100,        # Usamos 100 árboles
    max_depth=10,             # Profundidad máxima de los árboles
    learning_rate=0.1,       # Tasa de aprendizaje
    subsample=0.8,           # Proporción de muestras a usar
    colsample_bytree=0.8,    # Proporción de características a usar
    random_state=42,         # Fijamos una semilla para reproducibilidad
    n_jobs=-1                # Usamos todos los núcleos disponibles para el entrenamiento
)

# Entrenar el modelo
xgb_model_simple.fit(X_train_split, y_train_log)

# Predecir en el conjunto de validación
y_pred_xgb_simple_log = xgb_model_simple.predict(X_val_split)

# Invertir la transformación logarítmica para obtener los valores originales
y_pred_xgb_simple = np.expm1(y_pred_xgb_simple_log)

# Calcular el MAE
mae_xgb_simple = mean_absolute_error(y_val_split, y_pred_xgb_simple)

print(f'MAE del modelo XGBoost: {mae_xgb_simple}')


MAE del modelo XGBoost: 238.9112600391159


In [8]:
# Predecir en el conjunto de prueba
y_pred_test_log = xgb_model_simple.predict(X_test_improved)

# Invertir la transformación logarítmica para obtener los valores de precio originales
y_pred_test = np.expm1(y_pred_test_log)

# Crear un DataFrame con los resultados
output_df = test_df[['id']]
output_df['Price_euros'] = y_pred_test

# Guardar el archivo CSV de salida
output_df.to_csv('laptop_price_predictions.csv', index=False)
print('Archivo de predicciones guardado como laptop_price_predictions.csv')


Archivo de predicciones guardado como laptop_price_predictions.csv


C:\Users\mikel\AppData\Local\Temp\ipykernel_8504\3021490609.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['Price_euros'] = y_pred_test
